# 🔍 LightRAG Validation with Arize Phoenix

### Overview

This notebook provides one approach for validating and monitoring LightRAG's interaction with LLMs and embedding models -- leveraging [Arize Phoenix](https://docs.arize.com/phoenix/tracing/llm-traces-1) it provides insight into what is a very complex data ingestion pipeline.

It will also make the concepts covered in the LightRAG paper more tangible.

### Purpose
- **System Monitoring**: Validate LightRAG's integration with telemetry pipelines to ensure robust tracking of model inference and embedding use.
- **Performance Tuning**: Identify bottlenecks and optimize configurations using insights from telemetry data.
- **Proactive Debugging**: Quickly detect and resolve anomalies through real-time analysis.

### Key Features
- **Dockerized Deployment**: Simplifies setup with preconfigured Docker containers for Arize Phoenix.
- **Telemetry Integration**: Supports integration with external systems through use of OpenTelemetry standard to provide detailed system traces.
- **Customizable Dashboards**: Enables interactive exploration of model metrics and error logs.

### Usage Instructions
1. **Setup**: 
    - Install required dependencies:
      ```bash
      pip install arize-phoenix-otel
      ```
    - Run the Docker container for Arize Phoenix:
      ```bash
      docker run -p 6006:6006 -p 4317:4317 --rm arizephoenix/phoenix:latest
      ```

2. **Execute the Notebook**: Follow the provided steps in the notebook to validate your LightRAG setup against telemetry data.

3. **Explore Metrics**:
    - Access the Phoenix UI at [http://localhost:6006](http://localhost:6006).
    - Analyze detailed traces, latencies, and throughput metrics.

In [1]:
%pip install -q ipywidgets lightrag-hku openai aioboto3


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from pathlib import Path

# Define configuration constants
DATA_DIR = Path("../data")  # Base data directory
INTERIM_DIR = DATA_DIR / "interim"  # Interim data directory
PROCESSED_DIR = DATA_DIR / "processed"  # Processed data directory

In [3]:
import requests

# Define URL and local file path
src_file_url = "https://raw.githubusercontent.com/donbr/kg_rememberall/refs/heads/main/references/winston_churchill_we_shall_fight_speech_june_1940.txt"
file_name = src_file_url.split("/")[-1].replace(".", "_").lower()

WORKING_DIR = INTERIM_DIR / file_name

if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)

In [4]:
# Fetch and save the file
response = requests.get(src_file_url)
response.raise_for_status()  # Raise an exception for HTTP errors
local_file_path = WORKING_DIR / f"{file_name}.txt"
local_file_path.write_text(response.text)

# Define file paths
GRAPHML_FILE = WORKING_DIR / "graph_chunk_entity_relation.graphml"
PYVIS_HTML_FILE = PROCESSED_DIR / f"{file_name}.html"
PYVIS_HTML_FILE2 = PROCESSED_DIR / f"{file_name}2.html" 

## Arize Phoenix

- UI endpoint:  http://localhost:6006
- NOTE:  the Docker container will be removed when you shut down the notebook.

In [5]:
# for more information refer to https://docs.arize.com/phoenix/tracing/integrations-tracing/autogen-support#docker
# !docker run -p 6006:6006 -p 4317:4317 arizephoenix/phoenix:latest

import subprocess

# Run the Docker container without interactive mode
subprocess.Popen([
    "docker", "run", "-p", "6006:6006", "-p", "4317:4317",
    "--rm", "arizephoenix/phoenix:latest"
])

<Popen: returncode: None args: ['docker', 'run', '-p', '6006:6006', '-p', '4...>

🏃‍♀️‍➡️ Running migrations on the database.
---------------------------
2025-01-20 04:26:29,002 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-20 04:26:29,002 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
2025-01-20 04:26:29,002 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 04:26:29,003 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("alembic_version")
2025-01-20 04:26:29,003 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 04:26:29,003 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("alembic_version")
2025-01-20 04:26:29,003 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 04:26:29,003 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("alembic_version")
2025-01-20 04:26:29,003 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-20 04:26:29,003 INFO sqlalchemy.engine.Engine 
CREATE TABLE alembic_version (
	version_num VARCHAR(32) NOT NULL, 
	CONSTRAINT alembic_version_pkc PRIMARY KEY (version_num)
)


2025-01-20 04:26:29,0

INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:6006 (Press CTRL+C to quit)


## Arize Phoenix:  setup and configuration

In [6]:
%pip install -q arize-phoenix-otel


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
from phoenix.otel import register

# defaults to endpoint="http://localhost:4317"
tracer_provider = register(
  project_name="lightrag-openai", # Default is 'default'
  endpoint="http://localhost:4317",  # Sends traces using gRPC
)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: lightrag-openai
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'authorization': '****', 'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [8]:
## install python telemetry and openai library requirements
%pip install -q openinference-instrumentation-openai openai 'httpx<0.28'


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
from openinference.instrumentation.openai import OpenAIInstrumentor

OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

## Example Use Case

Monitor LightRAG’s real-time LLM and embedding model usage
- performance and response latencies
- model behavior and accuracy

### Populate the Graph

- Initialize LightRAG and OpenAI connection

In [10]:
# install Ollama as LightRAG requires it
%pip install -q ollama tiktoken nano_vectordb


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
from lightrag import LightRAG, QueryParam
from lightrag.llm import gpt_4o_mini_complete
import nano_vectordb

#########
# Uncomment the below two lines if running in a jupyter notebook to handle the async nature of rag.insert()
import nest_asyncio
nest_asyncio.apply()
#########

rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func=gpt_4o_mini_complete  # Use gpt_4o_mini_complete LLM model
    # llm_model_func=gpt_4o_complete  # Optionally, use a stronger model
)

INFO:lightrag:Logger initialized for working directory: ../data/interim/winston_churchill_we_shall_fight_speech_june_1940_txt
INFO:lightrag:Load KV llm_response_cache with 0 data
INFO:lightrag:Load KV full_docs with 0 data
INFO:lightrag:Load KV text_chunks with 0 data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': '../data/interim/winston_churchill_we_shall_fight_speech_june_1940_txt/vdb_entities.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': '../data/interim/winston_churchill_we_shall_fight_speech_june_1940_txt/vdb_relationships.json'} 0 data
INFO:nano-vectordb:Init {'embedding_dim': 1536, 'metric': 'cosine', 'storage_file': '../data/interim/winston_churchill_we_shall_fight_speech_june_1940_txt/vdb_chunks.json'} 0 data
INFO:lightrag:Loaded document status storage with 0 records


In [12]:
with open(local_file_path) as f:
    rag.insert(f.read())

INFO:lightrag:Processing 1 new unique documents
Processing batch 1:   0%|          | 0/1 [00:00<?, ?it/s]INFO:lightrag:Inserting 5 vectors to chunks
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.36s/batch]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Extracting entities from chunks: 100%|██████████| 5/5 [00:27<00:00,  5.53s/chunk]
INFO:lightrag:Inserting entities into storage...
Inserting entities: 100%|██████████| 91/91 [00:00<00:00, 27602.09entity/s]
INFO:lightrag:Inserting relationships into storage...
Inserting relationships: 100%|██████████| 36/36 [00:00<00:00, 12717.51relationship/s]
INFO:lightrag:Inserting 91 vectors to entities
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Generating embeddings: 100%|██████████| 3/3 [00:01<00:00,  1.89batch/s]
INFO:lightrag:Inserting 36 vectors to relationships
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Generating embeddings: 100%|██████████| 2/2 [00:01<00:00, 

### Query the Graph

In [13]:
# Perform hybrid search
print("\n## APPROACH 4\n")
print(rag.query("What role did King Leopold play during the evacuation of Dunkirk?", param=QueryParam(mode="hybrid")))


## APPROACH 4



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:lightrag:kw_prompt result:
INFO:lightrag:Using hybrid mode for query processing


{
  "high_level_keywords": ["King Leopold", "Evacuation of Dunkirk", "Historical role"],
  "low_level_keywords": ["World War II", "British Expeditionary Force", "French forces", "Military strategy", "Belgium"]
}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Local query uses 60 entites, 34 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:lightrag:Global query uses 45 entites, 35 relations, 3 text units
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


### King Leopold's Influence and Decisions

King Leopold of Belgium played a significant role during the events leading up to the evacuation of Dunkirk. His decisions directly influenced the fate of the Belgian Army and impacted military strategies for the Allied forces as they faced the German invasion during World War II. Initially, King Leopold sought assistance from the Allies when Belgium was invaded, which led to the involvement of British and French troops in an effort to defend Belgium. 

However, as the German forces advanced and the situation became increasingly dire, King Leopold made a critical decision that would have far-reaching implications. He unilaterally decided to surrender the Belgian Army to the Germans without prior consultation or the advice of his ministers. This act exposed the Allies' flank and left the British and French forces in a vulnerable position, impacting their ability to effectively retreat.

### Consequences of the Surrender

The surrender of the B

### Display the Graph

- graph visualization

In [14]:
import networkx as nx
from pyvis.network import Network
import json

def create_interactive_visualization(graphml_file, output_file):
    # Load the GraphML file
    G = nx.read_graphml(graphml_file)
    
    # Create Pyvis network
    net = Network(height='900px', width='100%', bgcolor='#ffffff', 
                 font_color='black', notebook=False)
    
    # Define color scheme for entity types
    entity_colors = {
        'PERSON': '#e41a1c',        # Bright red
        'ORGANIZATION': '#377eb8',   # Blue
        'GEO': '#4daf4a',           # Green
        'EVENT': '#984ea3',         # Purple
        'CONCEPT': '#ff7f00',       # Orange
        'TECHNOLOGY': '#a65628',     # Brown
        'CATEGORY': '#f781bf',      # Pink
        'NUMBER': '#999999',        # Gray
        'UNKNOWN': '#808080'        # Dark Gray
    }
    
    # Add nodes with colors before adding edges
    for node_id, node_data in G.nodes(data=True):
        # Get entity type (removing quotes if present)
        entity_type = node_data.get('entity_type', 'UNKNOWN').replace('"', '')
        color = entity_colors.get(entity_type, '#808080')
        
        # Create hover text
        hover_info = f"""
        Entity: {node_id}
        Type: {entity_type}
        Description: {node_data.get('description', 'N/A')}
        Source ID: {node_data.get('source_id', 'N/A')}
        """
        
        # Add node with properties
        net.add_node(node_id, 
                    title=hover_info,
                    color=color,
                    size=30)

    # Add edges
    for source, target, edge_data in G.edges(data=True):
        weight = edge_data.get('weight', 1)
        description = edge_data.get('description', '')
        
        hover_info = f"""
        Weight: {weight}
        Description: {description}
        Keywords: {edge_data.get('keywords', 'N/A')}
        """
        
        net.add_edge(source, target,
                    title=hover_info,
                    width=float(weight),
                    color={'color': '#666666', 'highlight': '#ff0000'})

    # Rest of your physics and legend code remains the same
    physics_options = {
        "physics": {
            "forceAtlas2Based": {
                "gravitationalConstant": -100,
                "centralGravity": 0.01,
                "springLength": 200,
                "springConstant": 0.08,
                "damping": 0.4,
                "avoidOverlap": 1
            },
            "solver": "forceAtlas2Based",
            "stabilization": {
                "enabled": True,
                "iterations": 1000,
                "updateInterval": 25
            }
        }
    }
    
    net.set_options(json.dumps(physics_options))
    
    # Save and add legend
    net.write_html(output_file)
    
    # Add legend HTML
    legend_html = """
    <div style="position: absolute; top: 10px; left: 10px; background-color: rgba(255, 255, 255, 0.9); 
                padding: 10px; border-radius: 5px; border: 1px solid #ccc;">
        <h3>Entity Types</h3>
        <ul style="list-style-type: none; padding: 0;">
    """
    
    for entity_type, color in entity_colors.items():
        legend_html += f"""
            <li style="margin: 5px 0;">
                <span style="display: inline-block; width: 20px; height: 20px; 
                           background-color: {color}; border-radius: 50%; margin-right: 5px;"></span>
                {entity_type}
            </li>
        """
    
    legend_html += """
        </ul>
    </div>
    """
    
    with open(output_file, 'r', encoding='utf-8') as file:
        content = file.read()
    content = content.replace('</body>', f'{legend_html}</body>')
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(content)

In [15]:
# Generate Visualization
create_interactive_visualization(GRAPHML_FILE, str(PYVIS_HTML_FILE))

## Review Telemetry data

- Access the Arize Phoenix UI at [http://localhost:6006](http://localhost:6006)
- both LLM inference and embedding telemetry information is captured